# 実習1　分類

## 実習概要

scikit-learnのライブラリを使用して、以下の操作を習得します。

- ロジスティック回帰による分類
- ランダムフォレストによる分類
- 最適な評価指標によるモデル評価

※セルは適宜追加してください。

## データ概要

- クレジットカード.csv

    C:\PyML\exercise\1.Classifier\クレジットカード.csv

    あなたはクレジットカード会社A社の運用担当です。将来的に債務不履行を起こす予兆があるユーザーを事前に把握し、アプローチしていきたいと考えています。

|列名|意味|
|:--|:--|
|性別、最終学歴など|クレジットカード利用者のユーザー情報|
|支払状況|過去の滞納情報。「なし」=滞納なし、「Nヶ月」=Nヶ月滞納。|
|請求額|クレジットカード会社からの請求額|
|支払額|クレジットカード会社からの貸し出し額|
|債務不履行|当月の債務不履行の発生有無（1が債務不履行あり、0が債務不履行なし）|

## 事前準備

### ライブラリのインポート

※必要であれば、ライブラリを追加インポートしてください

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,make_scorer

### ファイルの読み込み

In [2]:
df = pd.read_csv('クレジットカード.csv',encoding='shift_jis',engine='python')

In [3]:
df.head(3)

,性別,最終学歴,結婚,支払状況_1ヵ月前,支払状況_2ヵ月前,支払状況_3ヵ月前,支払状況_4ヵ月前,支払状況_5ヵ月前,支払状況_6ヵ月前,年齢,...,請求額_4ヵ月前,請求額_5ヵ月前,請求額_6ヵ月前,支払額_1ヵ月前,支払額_2ヵ月前,支払額_3ヵ月前,支払額_4ヵ月前,支払額_5ヵ月前,支払額_6ヵ月前,債務不履行
0,女,大学,既婚,2ヵ月,2ヵ月,なし,なし,なし,なし,24,...,0,0,0,0,689,0,0,0,0,1
1,女,大学,未婚,なし,2ヵ月,なし,なし,なし,2ヵ月,26,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,女,大学,未婚,なし,なし,なし,なし,なし,なし,34,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0


## データ加工

### 欠損値削除

In [4]:
df_post_drop = df.dropna()

In [5]:
df_post_drop.shape

(29931, 24)

In [6]:
df_post_drop = df_post_drop.reset_index(drop=True) # indexの振りなおしをしないと、後でconcatする際、正しく結合されないので注意

In [7]:
df_post_drop.head(1)

,性別,最終学歴,結婚,支払状況_1ヵ月前,支払状況_2ヵ月前,支払状況_3ヵ月前,支払状況_4ヵ月前,支払状況_5ヵ月前,支払状況_6ヵ月前,年齢,...,請求額_4ヵ月前,請求額_5ヵ月前,請求額_6ヵ月前,支払額_1ヵ月前,支払額_2ヵ月前,支払額_3ヵ月前,支払額_4ヵ月前,支払額_5ヵ月前,支払額_6ヵ月前,債務不履行
0,女,大学,既婚,2ヵ月,2ヵ月,なし,なし,なし,なし,24,...,0,0,0,0,689,0,0,0,0,1


### ホールドアウト法で分割

※本演習では、特徴変数としては、数値データを指定してください。

In [8]:
X = df_post_drop.iloc[:,9:-1].values
y = df_post_drop.iloc[:,-1].values

In [9]:
X_train , X_test , y_train , y_test = train_test_split(X, y, test_size = 0.5)

## 学習、評価

下記の手法を使用して予測モデルを作成し、評価してください。

- ロジスティック回帰（正則化弱め ※C=10以上）
- ロジスティック回帰（L1正則化強め ※C=1以下）
- ロジスティック回帰（L2正則化強め ※C=1以下）
- ランダムフォレスト

### 学習

#### ロジスティック回帰（正則化弱め ※C=10以上）

In [10]:
clf_lr = LogisticRegression(C=10)

In [11]:
clf_lr.fit(X = X_train, y = y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [12]:
clf_lr.coef_

array([[ -1.04553459e-02,  -3.68189497e-06,  -8.83380270e-06,
          3.41073164e-06,   4.73068547e-06,   4.91273034e-08,
          3.51797997e-06,   1.71622674e-06,  -3.30303559e-05,
         -2.78488244e-05,  -1.93012624e-05,  -8.75314972e-06,
         -3.12971484e-06,  -2.25641981e-06]])

#### ロジスティック回帰（L1正則化強め ※C=1以下）

In [13]:
clf_l1 = LogisticRegression(penalty='l1', C=0.001)

In [14]:
clf_l1.fit(X = X_train, y = y_train)

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
clf_l1.coef_

array([[ -1.47924452e-02,  -3.13598864e-06,  -8.14812402e-06,
          3.25649453e-06,   4.59331166e-06,   4.78920132e-10,
          3.26146672e-06,   1.71273877e-06,  -3.06266623e-05,
         -2.56852392e-05,  -1.80500586e-05,  -8.16364386e-06,
         -3.11676301e-06,  -2.41085282e-06]])

#### ロジスティック回帰（L2正則化強め ※C=1以下）

In [16]:
clf_l2 = LogisticRegression(penalty='l2', C=0.001)

In [17]:
clf_l2.fit(X = X_train, y = y_train)

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [18]:
clf_l2.coef_

array([[ -8.79228952e-03,  -3.89493054e-06,  -9.10129301e-06,
          3.44597601e-06,   4.81529299e-06,   6.92404919e-08,
          3.57681531e-06,   1.75174352e-06,  -3.40609969e-05,
         -2.87137731e-05,  -1.97961495e-05,  -8.93463458e-06,
         -3.18556511e-06,  -2.18041382e-06]])

#### ランダムフォレスト

In [19]:
clf_rf = RandomForestClassifier(n_estimators=2000, n_jobs=-1, random_state=1)

In [20]:
clf_rf.fit(X = X_train, y = y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=2000, n_jobs=-1, oob_score=False, random_state=1,
            verbose=0, warm_start=False)

### 評価

#### 評価用の関数

In [21]:
def eval(method,scorer):
    print('学習用データ:', scorer(y_pred=method.predict(X=X_train),y_true=y_train))
    print('テスト用データ:', scorer(y_pred=method.predict(X=X_test),y_true=y_test))

#### ロジスティック回帰（正則化なし）

In [22]:
eval(clf_lr,accuracy_score)

学習用データ: 0.777881724023
テスト用データ: 0.779032473607


#### ロジスティック回帰（L1正則化）

In [23]:
eval(clf_l1,accuracy_score)

学習用データ: 0.777948546609
テスト用データ: 0.779032473607


#### ロジスティック回帰（L2正則化）

In [24]:
eval(clf_l2,accuracy_score)

学習用データ: 0.777881724023
テスト用データ: 0.779032473607


#### ランダムフォレスト

In [25]:
eval(clf_rf,accuracy_score)

学習用データ: 0.997126628801
テスト用データ: 0.785714285714


accurarcy_scoreはそれなりに高いが、recall、precisionが低い。
※本当は債務不履行がある可能性がある人を見逃すモデルになってしまっている。